Source: https://python.langchain.com/docs/tutorials/llm_chain/

In [1]:
# Load the saved API keys

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BZplARjV1SZam8j94NHC8sNwduiDg', 'finish_reason': 'stop', 'logprobs': None}, id='run-0fa593de-882f-4c72-a29e-b4ff2b459bb2-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Exercise
Practice tweaking the above cells with different models, different SYSTEM messages, different Human messages

In [5]:
messages = [
    SystemMessage("Translate the following from English into Hindi"),
    HumanMessage("I like this cake."),
]

model.invoke(messages)

AIMessage(content='मुझे यह केक पसंद है।', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 23, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BZplLL2Y04sDYxG9kchf5MhZkW3WM', 'finish_reason': 'stop', 'logprobs': None}, id='run-471d39ff-0273-4151-9f03-2990cfbadd46-0', usage_metadata={'input_tokens': 23, 'output_tokens': 8, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
messages = [
    SystemMessage("Given the information about the school, tell me which schools is best."),
    HumanMessage("10 kids like to go to DPS school. Nobody likes Infinity"),
    HumanMessage("Which school is best."),
]

model.invoke(messages)

AIMessage(content='Based on the information provided, DPS school is preferred by the 10 kids, while nobody likes Infinity. Therefore, DPS school is the best choice among the two.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 46, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BZplTBfePmfYxyvNqErn1g10A10Uo', 'finish_reason': 'stop', 'logprobs': None}, id='run-86da05a3-142b-499a-afcc-1efd38312d6e-0', usage_metadata={'input_tokens': 46, 'output_tokens': 33, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Streaming Message

In [7]:
import time
for token in model.stream(messages):
    print(token.content, end="|")
    time.sleep(0.1)

|Based| on| the| information| provided|,| it| seems| that| DPS| school| is| preferred| by| |10| kids|,| while| no| one| likes| Infinity|.| Therefore|,| DPS| school| appears| to| be| the| best| option| based| on| this| preference|.||

## Prompt templates

In [8]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

print(prompt_template)

input_variables=['language', 'text'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following from English into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})]


In [9]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [10]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [11]:
response = model.invoke(prompt)
print(response.content)

Ciao!


# Structured Output

https://python.langchain.com/docs/how_to/structured_output/

In [12]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [13]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic outout structure
class JokeFormat(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(JokeFormat)

joke = structured_llm.invoke("Tell me a joke about cats")
print(joke)

setup='Why did the cat sit on the computer?' punchline='Because it wanted to keep an eye on the mouse!' rating=8


In [14]:
#example

from typing import Optional
from pydantic import BaseModel, Field

# Pydantic
class AnimalFormat(BaseModel):
    """General description about a typical animal"""
    weight: int = Field(description="average weight of the animal in kg")
    lifeExpectancy: int = Field(description="Average Life expectancy of the animal in years")


structured_llm = llm.with_structured_output(AnimalFormat)

output = structured_llm.invoke("Tell me about human")
print(output)

weight=62 lifeExpectancy=72


In [15]:
# exercise: rag query

# Exercise: TOY RAG prompts

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

# RAG external text
file = open(".\data\indianTales.txt")
text = file.read()
#text = text[-len(text)//10:]
#print(text)

# Pydantic
class BookSummary(BaseModel):
    """A model to summarize a story book"""
    summary: str = Field(description="A short summary of the book")
    listOfCharacters: list[str] = Field(description="List of characters in the book")



# example toy RAG
prompt = ChatPromptTemplate([prompt_template])
msg = prompt.invoke({"question":"Summerize the book and give a list of main characters in the story.", "context":text})


structured_model = model.with_structured_output(BookSummary)

output = structured_model.invoke(msg)
print(output)

summary='"Indian Fairy Tales" is a collection of traditional tales compiled by Joseph Jacobs, featuring stories rich in moral lessons, cultural elements, and magical encounters. The tales include various characters, such as clever animals, princes, and magical beings, who navigate challenges and showcase the importance of values like generosity, honesty, and wisdom. The book presents a diverse range of narratives influenced by Indian folklore, highlighting themes of friendship, loyalty, and the consequences of actions.' listOfCharacters=['Raja Salabhan', 'Queen Lona', 'Raja Rasalu', 'Brahma-datta', 'Devadatta', 'Raja Sarkap', 'Katar', 'The King', 'The Fakir', 'The Crow', 'The Pigeon', "The Gardener's Daughter", 'Harisarman']


In [16]:
# Exercise: Generate a list of question and answers based on a given text

prompt_template = """You are an assistant for a research scholar. Based on the given article, list major points of the article. Don't user prior knowledge.
Artile: {article} 
Major Points:"""

# RAG external text
file = open(".\data\\animalResearch.txt")
text = file.read()

# example toy RAG
prompt = ChatPromptTemplate([prompt_template])
msg = prompt.invoke({"article":text})

output = llm.invoke(msg)

In [17]:
print(output.content)

1. **Scale of Animal Testing**: Over 110 million animals are killed annually in U.S. laboratories, including a range of species such as mice, rats, dogs, cats, and monkeys, for various forms of research and testing.

2. **Animal Suffering**: Animals in laboratories often endure severe suffering from experiments, including exposure to toxic substances and physical harm, as well as psychological distress from confinement and isolation.

3. **Unreliability of Animal Experiments**: A significant percentage of U.S. adults oppose animal testing. Many animal experiments do not effectively contribute to human health advancements, raising questions about their reliability and value.

4. **Poor Translation to Human Health**: Research published in reputable journals indicates that treatments developed in animal models seldom translate effectively to humans, with numerous failed clinical trials stemming from animal testing.

5. **Cost and Failure Rates**: Drug development is lengthy and costly, wi

In [20]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [21]:
pages[12].page_content

'A KS THEOREM IN SEVERAL VARIABLES 13\n[2] A. M. Gleason. A characterization of maximal ideals. Journal d’Analyse Math´ ematique, 19:171–\n172, 1967.\n[3] Jaikishan, S. Lata, and D. Singh. Multiplicativity of linear functiona ls on function spaces on an\nopen disc. Archiv der Mathematik , 123:65–74, 2024.\n[4] J. Kahane and W. ˙Zelazko. A characterization of maximal ideals in commutative Banach alge-\nbras. Studia Mathematica , 29:339–343, 1968.\n[5] K. Kou and J. Liu. Which linear operators preserve outer functio ns? Indagationes Mathemati-\ncae, 28(6):1144–1156, 2017.\n[6] S. Kowalski and Z. S/suppress l odkowski. A characterization of multiplicative linear functionals in Banach\nalgebras. Studia Mathematica , 67:215–223, 1980.\n[7] A. Maouche. Formes multiplicatives ` a valeurs dans le spectre. Colloquium Mathematicum ,\n71(1):43–45, 1996.\n[8] J. Mashreghi and T. Ransford. A Gleason-Kahane-˙Zelazko theorem for modules and applications\nto holomorphic function spaces. Bulletin of th

# More Bigger Prompting

In [21]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [23]:
from langchain.chat_models import init_chat_model
#model = init_chat_model("llama3-70b-8192", model_provider="groq")
model = init_chat_model("gpt-4o-mini", model_provider="openai")

question_template = """Tell me a joke about: {topic}
Write the response in the following format:

# {{Title}}
--------
{{response}}
"""
question = question_template.format(topic="college")
question


'Tell me a joke about: college\nWrite the response in the following format:\n\n# {Title}\n--------\n{response}\n'

In [24]:
response =  model.invoke(question_template.format(topic="cats"))

In [25]:
from IPython.display import Markdown
Markdown(response.content)

# Feline Fine
--------
Why was the cat sitting on the computer?  
Because it wanted to keep an eye on the mouse!

## Example 

In [26]:
from langchain.chat_models import init_chat_model
#model = init_chat_model("llama3-70b-8192", model_provider="groq")
model = init_chat_model("gpt-4o-mini", model_provider="openai")

prompt = """You are a research assistant. You are tasked to find 4 subtopics for the research area {topic}. For each sub area, list 5 major research work done so far in this format:

## {{Sub topic 1}}
1. {{Work 1}}
2. {{Work 2}}
3. {{Work 3}}
4. {{Work 4}}
5. {{Work 6}}
----
## {{Sub topic 2}}
1. {{Work 1}}
2. {{Work 2}}
3. {{Work 3}}
4. {{Work 4}}
5. {{Work 6}}
----
...
## {{Sub topic 4}}
1. {{Work 1}}
2. {{Work 2}}
3. {{Work 3}}
4. {{Work 4}}
5. {{Work 6}}
 
"""

response = model.invoke(prompt.format(topic="Machine Learning in stock predictions"))
from IPython.display import Markdown
Markdown(response.content)

Certainly! Here are four subtopics in the field of Machine Learning for stock predictions, along with notable research works in each area:

## Time Series Analysis and Forecasting
1. **Bontempi, G., Taieb, S. B., & Le Borgne, Y. A. (2012)** - "Machine learning strategies for time series forecasting."
2. **Hyndman, R. J., & Athanasopoulos, G. (2018)** - "Forecasting: Principles and Practice," which covers a wide variety of time series forecasting methods.
3. **Zhang, G. P. (2003)** - "Time series forecasting using a hybrid ARIMA and neural network model." 
4. **Makridakis, S., & Hibon, M. (2000)** - "The M3-Competition: Results, conclusions and recommendations." 
5. **Yao, J., & Tan, J. (2009)** - "Forecasting stock market movement direction with support vector machine."

----
## Feature Engineering and Selection
1. **Ameen, A., & Azzam, R. (2020)** - "Feature Engineering for Stock Price Prediction Using Machine Learning Techniques."
2. **Kourentzes, N., & Petropoulos, F. (2015)** - "The role of feature selection in stock price forecasting."
3. **Gu, Q., & Wang, L. (2016)** - "Feature selection for stock price prediction in a multi-class classification setting." 
4. **Verma, O. P., & Kumar, A. (2018)** - "A study on the selection of relevant indicators for predicting stock price movements."
5. **Xing, Y., & Korki, S. (2018)** - "Deep feature selection: Theory and application to stock price prediction."

----
## Deep Learning Approaches
1. **Fischer, T., & Krauss, C. (2018)** - "Deep learning with long short-term memory networks for financial market predictions."
2. **Cho, K. et al. (2014)** - "Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation," often cited in financial predictions.
3. **He et al. (2019)** - "A LSTM-based method for stock price prediction."
4. **Zhang, Y., & Hu, Z. (2019)** - "Stock Price Prediction Using LSTM and Historical Data."
5. **Shah, J. (2020)** - "Application of CNN and LSTM Neural Networks for Stock Market Predictions."

----
## Reinforcement Learning Applications
1. **Moody, J., & Wu, L. (1997)** - "Genetic algorithms for stock trading systems."
2. **Zhang, F., & Huo, L. (2018)** - "Reinforcement learning for trading in financial markets."
3. **Deng, Y., & Li, Y. (2016)** - "Deep reinforcement learning for trading."
4. **Jiang, Z., & Liang, J. (2017)** - "Portfolio Management via Deep Reinforcement Learning."
5. **Kakushadze, Z. (2019)** - "The Supervised Reinforcement Learning Approach to Stock Trading."

These works reflect significant advancements and various methodologies used in the application of machine learning to stock predictions.

## Example with sources

In [27]:
prompt_template = """You are an expert technical writer. 
            
Your task is to create a short report on the topic {focus}. Divide the report into easily digestable sections. You have to use prior knowledge. While writing the report use the following instructions:
        
1. Create a report structure using markdown formatting: Create a minimum of 4 sections.
- Use ## for the section title
- Use ### for sub-section headers
        
2. Write the report following this structure:
a. Title (## header)
b. Summary (### header)
c. Sources (### header)

3. Make your title engaging based upon the focus area of the analyst: 
{focus}

4. Write a summary section:
- Set up summary with general background / context related to the focus area of the analyst
- Emphasize what is novel, interesting, or surprising about insights gathered from the interview
- Create a numbered list of source documents, as you use them
- Do not mention the names of interviewers or experts
- Aim for approximately 2000 words maximum
- Use numbered sources in your report (e.g., [1], [2]) based on information from source documents
        
5. Write a Sources section:
- Include all sources used in your report
- Provide full links to relevant websites or specific document paths
- Separate each source by a newline. Use two spaces at the end of each line to create a newline in Markdown.
- It will look like:

### Sources
- [1] Link or Document name
- [2] Link or Document name

6. Be sure to combine sources. For example this is not correct:

[3] https://ai.meta.com/blog/meta-llama-3-1/
[4] https://ai.meta.com/blog/meta-llama-3-1/

There should be no redundant sources. It should simply be:

[3] https://ai.meta.com/blog/meta-llama-3-1/
        
7. Final review:
- Ensure the report follows the required structure
- Include no preamble before the title of the report
- Check that all guidelines have been followed

"""



response = model.invoke(prompt_template.format(focus="Machine Learning in stock predictions"))
from IPython.display import Markdown
Markdown(response.content)

## Machine Learning in Stock Predictions

### Summary
The stock market has long been a subject of interest for analysts and investors alike, fueled by the quest to predict stock prices accurately in an increasingly volatile economic landscape. Traditional statistical methods often fall short when it comes to accounting for the complexity and non-linearity inherent in financial data. However, recent advancements in machine learning (ML) have opened new avenues for developing predictive models that leverage vast datasets to generate insights that were previously unattainable. 

Incorporating techniques such as neural networks, decision trees, and ensemble methods, machine learning can analyze historical stock prices, trading volumes, and even social media sentiment to make predictions with remarkable accuracy. One particularly interesting aspect of machine learning in finance is the ability to adapt models in real-time as new data becomes available. This agility can be instrumental in responding to market shifts and investor behavior.

Recent insights gathered from the exploration of machine learning applications in stock prediction highlight the importance of feature engineering and the selection of algorithms tailored to specific problems. The novelty lies in how hybrid models, which combine fundamental analysis with technical data and sentiment analysis, can outperform traditional models in both short-term and long-term predictions. 

1. The financial sector has increasingly adopted machine learning techniques, pointing to a shift from traditional statistical methods to more data-driven approaches.
2. Hybrid models leveraging various data sources, including unstructured data, have shown significant improvements in predictive accuracy.
3. Ongoing research is focusing on interpretability in ML models, providing transparency and building trust with investors.

### Sources
- [1] https://www.forbes.com/sites/bernardmarr/2020/01/06/how-machine-learning-is-revolutionizing-stock-market-predictions/  
- [2] https://towardsdatascience.com/machine-learning-in-stock-price-prediction-832befcb0e56  
- [3] https://www.investopedia.com/terms/m/machine-learning.asp  
- [4] https://www.sciencedirect.com/science/article/pii/S2405452621001624  
- [5] https://arxiv.org/abs/1902.07985  

In [28]:
prompt_template="""
Write a detailed technical report on the mathematical topic: {topic}.

Structure the report into the following sections, each clearly separated with appropriate Markdown headings (#, ##):

# Introduction

{{Give a clear, concise overview of the topic.}}

{{Explain its significance and typical applications.}}

# Mathematical Theory

{{Describe the core mathematical foundations in detail.}}

{{Include important definitions, theorems, proofs, and examples where relevant.}}

# Latest Open Problems

{{Discuss major current research challenges or unresolved questions in the field.}}

# Reference recent papers or developments (with citations).

# Future Directions

{{Suggest possible paths for future research or applications.}}

{{Discuss emerging trends and technologies that might impact this area.}}

# References

List all sources cited, preferably in a simple numbered list or BibTeX-style.

Use appropriate LaTeX-style formatting (with $$ for equations if needed) for mathematical expressions inside the Markdown.
Write in a formal and precise tone, suitable for a technical audience.
"""


response = model.invoke(prompt_template.format(topic="Machine Learning in stock predictions"))
from IPython.display import Markdown
Markdown(response.content)

# Introduction

Machine learning (ML) has rapidly emerged as a powerful tool in quantitative finance for stock prediction. Stock prediction involves forecasting the future price movements of stocks based on historical data and various market indicators. The significance of ML in this domain lies in its ability to analyze vast amounts of data, identify complex patterns, and make informed trading decisions with minimal human intervention. Typical applications of ML in stock prediction include algorithmic trading, risk management, portfolio optimization, and sentiment analysis derived from news and social media.

# Mathematical Theory

## Core Mathematical Foundations

The mathematical foundations of machine learning in stock predictions encompass several areas, primarily statistics, linear algebra, optimization, and probability theory.

### Definitions and Notations

1. **Dataset**: A collection of financial data points is often represented as a matrix \( X \), where each row corresponds to an observation (i.e., a stock's attributes over time), and each column represents a feature (such as historical prices, volume, or technical indicators).

2. **Target Variable**: The outcome we wish to predict, denoted as \( y \), which could represent the future price of a stock or binary outcomes like "buy" or "sell".

3. **Model**: A function \( f: X \rightarrow y \) that attempts to map feature inputs to target outputs.

### Important Theorems and Concepts

1. **Empirical Risk Minimization**: This framework aims to find a function \( f \) that minimizes the empirical risk, defined as:

   $$
   R_n(f) = \frac{1}{n} \sum_{i=1}^{n} L(f(x_i), y_i)
   $$

   where \( L \) is a loss function quantifying the error between predicted and actual values.

2. **Overfitting and Regularization**: A critical challenge in ML models is overfitting, where the model performs well on training data but poorly on unseen data. Regularization techniques, such as Lasso \( (L_1 \text{ norm}) \) and Ridge \( (L_2 \text{ norm}) \), are used to constrain the model complexity and improve generalization.

3. **Gradient Descent**: An optimization method to minimize the loss function, iteratively updating the model parameters \( \theta \):

   $$
   \theta = \theta - \alpha \nabla L(\theta)
   $$

   where \( \alpha \) is the learning rate, and \( \nabla L \) is the gradient of the loss.

### Examples

For instance, a common ML model utilized is the **Support Vector Machine (SVM)**, which classifies data by finding the hyperplane that maximizes the margin between distinct classes. The mathematical formulation involves finding:

   $$
   \min_{\theta} \|\theta\|^2 \quad \text{subject to} \quad y_i(\langle \theta, x_i \rangle) \geq 1, \forall i
   $$

### Time Series Analysis

Since stock market data is inherently temporal, time series analysis methods, such as ARIMA (AutoRegressive Integrated Moving Average) and LSTM (Long Short-Term Memory networks), are essential. LSTMs, in particular, leverage recurrent neural network architecture to learn from sequences of data effectively, making them suitable for stock price prediction.

# Latest Open Problems

As machine learning continues to evolve in the context of stock prediction, several major research challenges remain:

1. **Interpretability of Models**: Complex models (like deep learning architectures) often function as "black boxes," making it difficult for traders to understand the reasoning behind predictions.

2. **Data Quality and Preprocessing**: Stock market data can be noisy and sparse. Developing robust methods for data cleaning, feature extraction, and transformation remains an active area of research.

3. **Dynamic Market Conditions**: Financial markets are influenced by diverse and changing factors, leading to non-stationary time series. Adapting models to such varying conditions without degrading predictive performance is a challenging task.

4. **Integration of Alternative Data**: There is considerable potential in incorporating non-traditional data sources, such as social media sentiment, financial news articles, and macroeconomic indicators, into stock prediction models. However, effectively modeling these data types poses its own set of complexities.

# Future Directions

Machine learning in stock predictions is poised for continual expansion, propelled by emerging trends and technologies. Possible paths for future research and application development include:

1. **Explainable AI (XAI)**: Innovations in making machine learning models more interpretable will be crucial for their adoption in finance, where stakeholders demand transparency and reliability in decision-making.

2. **Ensemble Methods**: Combining multiple models (e.g., Random Forests, Gradient Boosting) could yield better predictive performance than singular models due to enhanced robustness and accuracy.

3. **Reinforcement Learning**: This area can offer significant improvements in trading strategies by optimizing sequential decision-making processes, adapting to evolving market landscapes in real-time.

4. **Quantum Computing**: Future research could potentially explore the implications of quantum computing on machine learning algorithms, which could revolutionize processing capabilities for vast financial datasets.

5. **Integration with Blockchain**: Understanding the interactions between blockchain technology and stock prediction models could lead to novel insights and trading strategies.

# References

1. Tsay, R. S. (2010). *Analysis of Financial Time Series*. Wiley Series in Probability and Statistics.
2. He, K., Zhang, X., Ren, S., & Sun, J. (2016). "Deep Residual Learning for Image Recognition." *Proceedings of the IEEE Computer Society Conference on Computer Vision and Pattern Recognition*.
3. Kelleher, J. D., Mac Carthy, M. (2015). *Fundamentals of Machine Learning for Predictive Data Analytics: Algorithms, Worked Examples, and Case Studies*.
4. Hashem, I. A. T., et al. (2019). "A review of machine learning in stock market prediction." *Econometrics*.
5. Goodfellow, I., Bengio, Y., Courville, A. (2016). *Deep Learning*. MIT Press.